## 导入数据

In [1]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))


## 构建one-shot向量， 创建词集

In [2]:
vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

## 加入嵌入层，将矩阵乘法变为矩阵选择对应行相加

### 数据初始化

In [3]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weight_0_1 = 0.2 * np.random.random((len(vocab), hidden_size)) - 0.1
weight_1_2 = 0.2 * np.random.random((hidden_size, 1))  - 0.1


In [4]:
x = input_dataset[0]
weight_0_1[x].shape
x1 =  x[0]
print(weight_0_1[x][0])
print(weight_0_1[x1][0])

[ 0.05219542 -0.01003206  0.07806739 -0.05814723 -0.08462131  0.086042
  0.07659895 -0.01783922  0.04888394 -0.02254988  0.08850316 -0.08439556
  0.03936835 -0.04991844  0.00930056 -0.02473473  0.08152517 -0.00190765
 -0.05666182 -0.08074742 -0.02530067 -0.00051822 -0.01057741 -0.08571923
 -0.09776355  0.05655823  0.0917885  -0.04789848 -0.03991819  0.03752338
  0.03747385 -0.05989932 -0.05098615 -0.02699979  0.02075631  0.09090745
  0.04448967 -0.06647163  0.01790139 -0.05875186 -0.06355213  0.08773072
  0.0053167  -0.0531403  -0.06766583 -0.04528934 -0.0753661   0.06359688
 -0.06840724 -0.09327499 -0.01805317  0.00833489 -0.06959733  0.00135972
  0.04644753  0.05415903  0.01819298  0.09166429  0.09623166  0.07457304
 -0.03186279  0.01105468  0.05475519 -0.00795226 -0.07788968  0.027042
  0.01624662 -0.04014809 -0.08210575  0.04902503 -0.0277038  -0.03446648
 -0.09085808 -0.04995741  0.053536   -0.09776939  0.08296442  0.01848132
  0.03376195 -0.0914398  -0.04444798 -0.07834359  0.003

In [5]:
correct, total = 0, 0
for iteration in range(iterations):
    for i in range(len(input_dataset) - 1000):
        x = input_dataset[i]
        y = target_dataset[i]
        # 选出输入节点相应的行 按列求和
        layer_1 = sigmoid(np.sum(weight_0_1[x], axis=0))
        layer_2 = sigmoid(np.dot(layer_1, weight_1_2))
        
        # 计算误差 
        layer_2_delta = layer_2 - y
        # 反向传播
        layer_1_delta = np.dot(layer_2_delta, weight_1_2.T)
        
        weight_0_1[x] -= alpha * layer_1_delta
        weight_1_2 -= alpha * np.outer(layer_1, layer_2_delta)
        
        if np.abs(layer_2_delta) < 0.5:
            correct += 1
            pass
        total += 1
        if i % 10 == 9:
            print('I: %d, progress: %f, correct rate: %f' %(i, i / float(len(input_dataset) - 1000), correct / float(total)))

I: 9, progress: 0.000375, correct rate: 0.200000
I: 19, progress: 0.000792, correct rate: 0.150000
I: 29, progress: 0.001208, correct rate: 0.200000
I: 39, progress: 0.001625, correct rate: 0.200000
I: 49, progress: 0.002042, correct rate: 0.200000
I: 59, progress: 0.002458, correct rate: 0.216667
I: 69, progress: 0.002875, correct rate: 0.242857
I: 79, progress: 0.003292, correct rate: 0.262500
I: 89, progress: 0.003708, correct rate: 0.255556
I: 99, progress: 0.004125, correct rate: 0.250000
I: 109, progress: 0.004542, correct rate: 0.245455
I: 119, progress: 0.004958, correct rate: 0.233333
I: 129, progress: 0.005375, correct rate: 0.238462
I: 139, progress: 0.005792, correct rate: 0.242857
I: 149, progress: 0.006208, correct rate: 0.246667
I: 159, progress: 0.006625, correct rate: 0.262500
I: 169, progress: 0.007042, correct rate: 0.258824
I: 179, progress: 0.007458, correct rate: 0.261111
I: 189, progress: 0.007875, correct rate: 0.268421
I: 199, progress: 0.008292, correct rate: 

I: 1759, progress: 0.073292, correct rate: 0.623864
I: 1769, progress: 0.073708, correct rate: 0.624294
I: 1779, progress: 0.074125, correct rate: 0.625843
I: 1789, progress: 0.074542, correct rate: 0.626257
I: 1799, progress: 0.074958, correct rate: 0.627778
I: 1809, progress: 0.075375, correct rate: 0.627624
I: 1819, progress: 0.075792, correct rate: 0.625275
I: 1829, progress: 0.076208, correct rate: 0.625683
I: 1839, progress: 0.076625, correct rate: 0.626630
I: 1849, progress: 0.077042, correct rate: 0.628108
I: 1859, progress: 0.077458, correct rate: 0.629032
I: 1869, progress: 0.077875, correct rate: 0.630481
I: 1879, progress: 0.078292, correct rate: 0.631383
I: 1889, progress: 0.078708, correct rate: 0.631746
I: 1899, progress: 0.079125, correct rate: 0.633158
I: 1909, progress: 0.079542, correct rate: 0.635079
I: 1919, progress: 0.079958, correct rate: 0.635417
I: 1929, progress: 0.080375, correct rate: 0.635233
I: 1939, progress: 0.080792, correct rate: 0.636598
I: 1949, pro

I: 3969, progress: 0.165375, correct rate: 0.724433
I: 3979, progress: 0.165792, correct rate: 0.725126
I: 3989, progress: 0.166208, correct rate: 0.725564
I: 3999, progress: 0.166625, correct rate: 0.726000
I: 4009, progress: 0.167042, correct rate: 0.726185
I: 4019, progress: 0.167458, correct rate: 0.726617
I: 4029, progress: 0.167875, correct rate: 0.726799
I: 4039, progress: 0.168292, correct rate: 0.726485
I: 4049, progress: 0.168708, correct rate: 0.726914
I: 4059, progress: 0.169125, correct rate: 0.727094
I: 4069, progress: 0.169542, correct rate: 0.727027
I: 4079, progress: 0.169958, correct rate: 0.726961
I: 4089, progress: 0.170375, correct rate: 0.727628
I: 4099, progress: 0.170792, correct rate: 0.728049
I: 4109, progress: 0.171208, correct rate: 0.728467
I: 4119, progress: 0.171625, correct rate: 0.728398
I: 4129, progress: 0.172042, correct rate: 0.729056
I: 4139, progress: 0.172458, correct rate: 0.729227
I: 4149, progress: 0.172875, correct rate: 0.729880
I: 4159, pro

I: 5929, progress: 0.247042, correct rate: 0.753963
I: 5939, progress: 0.247458, correct rate: 0.754040
I: 5949, progress: 0.247875, correct rate: 0.754454
I: 5959, progress: 0.248292, correct rate: 0.754195
I: 5969, progress: 0.248708, correct rate: 0.754439
I: 5979, progress: 0.249125, correct rate: 0.754348
I: 5989, progress: 0.249542, correct rate: 0.754424
I: 5999, progress: 0.249958, correct rate: 0.754500
I: 6009, progress: 0.250375, correct rate: 0.754576
I: 6019, progress: 0.250792, correct rate: 0.754817
I: 6029, progress: 0.251208, correct rate: 0.754892
I: 6039, progress: 0.251625, correct rate: 0.754305
I: 6049, progress: 0.252042, correct rate: 0.754380
I: 6059, progress: 0.252458, correct rate: 0.753960
I: 6069, progress: 0.252875, correct rate: 0.754036
I: 6079, progress: 0.253292, correct rate: 0.754276
I: 6089, progress: 0.253708, correct rate: 0.754516
I: 6099, progress: 0.254125, correct rate: 0.754754
I: 6109, progress: 0.254542, correct rate: 0.754992
I: 6119, pro

I: 8229, progress: 0.342875, correct rate: 0.783597
I: 8239, progress: 0.343292, correct rate: 0.783617
I: 8249, progress: 0.343708, correct rate: 0.783515
I: 8259, progress: 0.344125, correct rate: 0.783656
I: 8269, progress: 0.344542, correct rate: 0.783555
I: 8279, progress: 0.344958, correct rate: 0.783816
I: 8289, progress: 0.345375, correct rate: 0.784077
I: 8299, progress: 0.345792, correct rate: 0.784337
I: 8309, progress: 0.346208, correct rate: 0.784597
I: 8319, progress: 0.346625, correct rate: 0.784615
I: 8329, progress: 0.347042, correct rate: 0.784754
I: 8339, progress: 0.347458, correct rate: 0.784892
I: 8349, progress: 0.347875, correct rate: 0.784910
I: 8359, progress: 0.348292, correct rate: 0.784689
I: 8369, progress: 0.348708, correct rate: 0.784707
I: 8379, progress: 0.349125, correct rate: 0.784606
I: 8389, progress: 0.349542, correct rate: 0.784744
I: 8399, progress: 0.349958, correct rate: 0.784881
I: 8409, progress: 0.350375, correct rate: 0.784780
I: 8419, pro

...

I: 14059, progress: 0.585792, correct rate: 0.857987
I: 14069, progress: 0.586208, correct rate: 0.857972
I: 14079, progress: 0.586625, correct rate: 0.858009
I: 14089, progress: 0.587042, correct rate: 0.857942
I: 14099, progress: 0.587458, correct rate: 0.857953
I: 14109, progress: 0.587875, correct rate: 0.857964
I: 14119, progress: 0.588292, correct rate: 0.858001
I: 14129, progress: 0.588708, correct rate: 0.858038
I: 14139, progress: 0.589125, correct rate: 0.858023
I: 14149, progress: 0.589542, correct rate: 0.858034
I: 14159, progress: 0.589958, correct rate: 0.858045
I: 14169, progress: 0.590375, correct rate: 0.858056
I: 14179, progress: 0.590792, correct rate: 0.858067
I: 14189, progress: 0.591208, correct rate: 0.858078
I: 14199, progress: 0.591625, correct rate: 0.858115
I: 14209, progress: 0.592042, correct rate: 0.858126
I: 14219, progress: 0.592458, correct rate: 0.858111
I: 14229, progress: 0.592875, correct rate: 0.858122
I: 14239, progress: 0.593292, correct rate: 0.

I: 16229, progress: 0.676208, correct rate: 0.859906
I: 16239, progress: 0.676625, correct rate: 0.859940
I: 16249, progress: 0.677042, correct rate: 0.859975
I: 16259, progress: 0.677458, correct rate: 0.859960
I: 16269, progress: 0.677875, correct rate: 0.859995
I: 16279, progress: 0.678292, correct rate: 0.860030
I: 16289, progress: 0.678708, correct rate: 0.860040
I: 16299, progress: 0.679125, correct rate: 0.860050
I: 16309, progress: 0.679542, correct rate: 0.859985
I: 16319, progress: 0.679958, correct rate: 0.859995
I: 16329, progress: 0.680375, correct rate: 0.859955
I: 16339, progress: 0.680792, correct rate: 0.859965
I: 16349, progress: 0.681208, correct rate: 0.859975
I: 16359, progress: 0.681625, correct rate: 0.860010
I: 16369, progress: 0.682042, correct rate: 0.860020
I: 16379, progress: 0.682458, correct rate: 0.860030
I: 16389, progress: 0.682875, correct rate: 0.860040
I: 16399, progress: 0.683292, correct rate: 0.860000
I: 16409, progress: 0.683708, correct rate: 0.

I: 18429, progress: 0.767875, correct rate: 0.862244
I: 18439, progress: 0.768292, correct rate: 0.862253
I: 18449, progress: 0.768708, correct rate: 0.862238
I: 18459, progress: 0.769125, correct rate: 0.862270
I: 18469, progress: 0.769542, correct rate: 0.862279
I: 18479, progress: 0.769958, correct rate: 0.862312
I: 18489, progress: 0.770375, correct rate: 0.862321
I: 18499, progress: 0.770792, correct rate: 0.862329
I: 18509, progress: 0.771208, correct rate: 0.862362
I: 18519, progress: 0.771625, correct rate: 0.862371
I: 18529, progress: 0.772042, correct rate: 0.862403
I: 18539, progress: 0.772458, correct rate: 0.862412
I: 18549, progress: 0.772875, correct rate: 0.862421
I: 18559, progress: 0.773292, correct rate: 0.862383
I: 18569, progress: 0.773708, correct rate: 0.862391
I: 18579, progress: 0.774125, correct rate: 0.862424
I: 18589, progress: 0.774542, correct rate: 0.862456
I: 18599, progress: 0.774958, correct rate: 0.862418
I: 18609, progress: 0.775375, correct rate: 0.

I: 20639, progress: 0.859958, correct rate: 0.864830
I: 20649, progress: 0.860375, correct rate: 0.864770
I: 20659, progress: 0.860792, correct rate: 0.864778
I: 20669, progress: 0.861208, correct rate: 0.864719
I: 20679, progress: 0.861625, correct rate: 0.864749
I: 20689, progress: 0.862042, correct rate: 0.864735
I: 20699, progress: 0.862458, correct rate: 0.864743
I: 20709, progress: 0.862875, correct rate: 0.864751
I: 20719, progress: 0.863292, correct rate: 0.864781
I: 20729, progress: 0.863708, correct rate: 0.864789
I: 20739, progress: 0.864125, correct rate: 0.864819
I: 20749, progress: 0.864542, correct rate: 0.864849
I: 20759, progress: 0.864958, correct rate: 0.864857
I: 20769, progress: 0.865375, correct rate: 0.864865
I: 20779, progress: 0.865792, correct rate: 0.864895
I: 20789, progress: 0.866208, correct rate: 0.864925
I: 20799, progress: 0.866625, correct rate: 0.864911
I: 20809, progress: 0.867042, correct rate: 0.864896
I: 20819, progress: 0.867458, correct rate: 0.

I: 22679, progress: 0.944958, correct rate: 0.866153
I: 22689, progress: 0.945375, correct rate: 0.866181
I: 22699, progress: 0.945792, correct rate: 0.866210
I: 22709, progress: 0.946208, correct rate: 0.866238
I: 22719, progress: 0.946625, correct rate: 0.866224
I: 22729, progress: 0.947042, correct rate: 0.866253
I: 22739, progress: 0.947458, correct rate: 0.866260
I: 22749, progress: 0.947875, correct rate: 0.866289
I: 22759, progress: 0.948292, correct rate: 0.866275
I: 22769, progress: 0.948708, correct rate: 0.866282
I: 22779, progress: 0.949125, correct rate: 0.866246
I: 22789, progress: 0.949542, correct rate: 0.866253
I: 22799, progress: 0.949958, correct rate: 0.866282
I: 22809, progress: 0.950375, correct rate: 0.866311
I: 22819, progress: 0.950792, correct rate: 0.866318
I: 22829, progress: 0.951208, correct rate: 0.866346
I: 22839, progress: 0.951625, correct rate: 0.866354
I: 22849, progress: 0.952042, correct rate: 0.866382
I: 22859, progress: 0.952458, correct rate: 0.

## 测试

In [6]:
correct, total = (0, 0)
for i in range(len(input_dataset) - 1000, len(input_dataset)):
    x = input_dataset[i]
    y = target_dataset[i]
    
    layer_1 = sigmoid(np.sum(weight_0_1[x], axis=0))
    layer_2 = sigmoid(np.dot(layer_1, weight_1_2))
    
    layer_2_delta = layer_2 - y
    if np.abs(layer_2_delta) < 0.5:
        correct += 1
        pass
    total += 1
    
    if i % 10 == 9:
        print('I: %d, progress: %f, correct rate: %f, correct num: %d' %(i, i / float(len(input_dataset) - 1000), correct / float(total), correct))

I: 24009, progress: 1.000375, correct rate: 0.900000, correct num: 9
I: 24019, progress: 1.000792, correct rate: 0.900000, correct num: 18
I: 24029, progress: 1.001208, correct rate: 0.933333, correct num: 28
I: 24039, progress: 1.001625, correct rate: 0.925000, correct num: 37
I: 24049, progress: 1.002042, correct rate: 0.940000, correct num: 47
I: 24059, progress: 1.002458, correct rate: 0.950000, correct num: 57
I: 24069, progress: 1.002875, correct rate: 0.928571, correct num: 65
I: 24079, progress: 1.003292, correct rate: 0.912500, correct num: 73
I: 24089, progress: 1.003708, correct rate: 0.911111, correct num: 82
I: 24099, progress: 1.004125, correct rate: 0.910000, correct num: 91
I: 24109, progress: 1.004542, correct rate: 0.890909, correct num: 98
I: 24119, progress: 1.004958, correct rate: 0.891667, correct num: 107
I: 24129, progress: 1.005375, correct rate: 0.900000, correct num: 117
I: 24139, progress: 1.005792, correct rate: 0.892857, correct num: 125
I: 24149, progress

### 单词嵌入表达的对比，可视化权重的相似度

In [7]:
from collections import Counter
import math 

def similar(target='beautiful'):
    target_index = word2index[target]
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weight_0_1[index] - (weight_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))

    return scores.most_common(10)

In [8]:
print(similar('beautiful'))

[('beautiful', -0.0), ('knowing', -0.7042338502199265), ('episode', -0.7494847994074334), ('atmosphere', -0.7646670029631445), ('unique', -0.7723952300100524), ('definitely', -0.7765847731209196), ('cry', -0.7779165517441501), ('fantastic', -0.7810207942079931), ('freedom', -0.7854604973582515), ('masterpiece', -0.7899362730378421)]


In [9]:
print(similar('terrible'))

[('terrible', -0.0), ('worse', -0.7900818750334466), ('lacks', -0.7903696439523438), ('fails', -0.7968572236838813), ('laughable', -0.8010459913344223), ('annoying', -0.8013341666405384), ('boring', -0.803207226923477), ('disappointing', -0.8138127753701314), ('avoid', -0.8210624608954272), ('poor', -0.8323490924415878)]


## 完型填空

In [10]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [11]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

def similar(target='beautiful'):
    target_index = word2index[target]

    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - (weights_0_1[target_index])
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
    return scores.most_common(10)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

for rev_i,review in enumerate(input_dataset * iterations):
    for target_i in range(len(review)):
        
        # since it's really expensive to predict every vocabulary
        # we're only going to predict a random subset
        target_samples = [review[target_i]]+list(concatenated\
        [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

        left_context = review[max(0,target_i-window):target_i]
        right_context = review[target_i+1:min(len(review),target_i+window)]

        layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
        layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
        layer_2_delta = layer_2 - layer_2_target
        layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

        weights_0_1[left_context+right_context] -= layer_1_delta * alpha
        weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha
#     if(rev_i % 250 == 0):
#         sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
#             *iterations)) + "   " + str(similar('terrible')))
#     sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
#             *iterations)))
print(similar('terrible'))

[('terrible', -0.0), ('horrible', -2.8649818080049387), ('brilliant', -3.469877091143049), ('pathetic', -3.7566393039566837), ('superb', -3.818909183516178), ('phenomenal', -3.932846177771029), ('masterful', -3.999621453481043), ('bad', -4.023898812536258), ('horrendous', -4.216895253486791), ('lame', -4.282385484020378)]


## King - Man + Woman ~= Queen

In [12]:
def analogy(positive=['terrible','good'],negative=['bad']):
    
    norms = np.sum(weights_0_1 * weights_0_1,axis=1)
    norms.resize(norms.shape[0],1)
    
    normed_weights = weights_0_1 * norms
    
    query_vect = np.zeros(len(weights_0_1[0]))
    for word in positive:
        query_vect += normed_weights[word2index[word]]
    for word in negative:
        query_vect -= normed_weights[word2index[word]]
    
    scores = Counter()
    for word,index in word2index.items():
        raw_difference = weights_0_1[index] - query_vect
        squared_difference = raw_difference * raw_difference
        scores[word] = -math.sqrt(sum(squared_difference))
        
    return scores.most_common(10)[1:]

In [13]:
analogy(['terrible','good'],['bad'])

[('superb', -225.08276465532305),
 ('terrific', -225.28350441900352),
 ('worth', -225.3546861082853),
 ('fine', -225.4690582919037),
 ('decent', -225.51337395622315),
 ('perfect', -225.88637518123033),
 ('nice', -225.9356928778446),
 ('brilliant', -225.9440632230949),
 ('outstanding', -225.98178708424803)]

In [14]:
analogy(['elizabeth','he'],['she'])

[('christopher', -194.43500960997488),
 ('tom', -194.44655342263786),
 ('david', -194.59427735238964),
 ('peter', -194.7040624914905),
 ('william', -194.82603440555337),
 ('this', -194.8387087033333),
 ('it', -194.86370953413808),
 ('fred', -194.8917369985579),
 ('mr', -194.95168921192595)]

## 平均词向量来预测两个句子的相关性

In [80]:
# 权重正则化
# 平方求和
norms = np.sum(weights_0_1 * weights_0_1,axis=1)
norms.resize(norms.shape[0], 1)
print(weights_0_1.shape)
print(norms.shape)
normed_weights = weights_0_1 * norms
normed_weights.shape

(74075, 50)
(74075, 1)


(74075, 50)

In [81]:
# 将句子向量化
def make_sent_vect(words):
    index = list()
    # 选出句子中存在于词集的词的下标
    for word in words:
        for w, i in word2index.items():
            if word == w:
                index.append(i)
                pass
            pass
        pass
    wordvec = np.mean(normed_weights[index], axis=0)
    return wordvec 

In [82]:
make_sent_vect(['boring', 'awful'])

array([ 27.5114651 , -22.18406886, -13.48282569,   5.2771248 ,
        -6.58696077, -16.64628298, -19.12965093,  34.56951064,
        14.70764394, -12.15205212,  19.31038972,  19.85357821,
        27.27990026,  -7.52131942,   5.17704623, -17.25977695,
        -7.75820383,  -9.70117102,  35.88115547, -35.17384187,
        -9.00949624,   8.23268002,  -9.35761684, -39.18081082,
        -8.28271356,   7.02500974, -32.37084312,   6.44985607,
         0.14428819,  -5.4179111 ,  24.17904116,  29.90574549,
         8.85524487, -30.43304265, -11.08288465, -30.86787536,
       -14.60959934, -15.26211505, -32.8370357 ,   4.07495261,
         6.92531154,  20.09501062, -11.75862608,  17.74073488,
        27.74592149,  43.95428186,   5.72188173, -36.47888237,
        -2.35799319,  11.92109979])

In [85]:
review2vec = list()
# 计算数据集所有句子的向量
for view in tokens[0:100]:
    # TODO 选取前100个评论
    review2vec.append(make_sent_vect(view))
    pass
review2vec

[array([  9.42325233,  -5.77066127,   4.90095176,  -3.62928542,
         13.80993171,   2.86051469,  -7.87496901,  -4.93757123,
          9.24850186,  -1.8628181 ,   1.66525164,  -4.03976957,
          3.01614987,   3.1142054 ,   4.31981803,  10.12158065,
         11.35541428,   1.16272188,   0.6329952 ,  -3.02653689,
         -1.75860346,   9.50723621,  20.08326208,  -2.20563566,
         -3.81731905,  -7.69622646,  -1.37353841,   4.11792133,
         11.07505305,  -2.21230174,   1.47693529,   9.3349815 ,
          6.60779044, -11.66253126,   2.34709566,  10.02390635,
         -8.8876534 ,   0.41422056, -12.44304305,  -8.2859568 ,
          5.48549393,  -3.99517603,  -4.4049386 ,  -0.79940868,
         -1.1254354 ,  16.56826159,   6.69680354, -12.82229271,
         19.68885716,  11.48831881]),
 array([ 1.15619071e+01, -4.55037668e+00,  5.61695255e+00, -1.02706450e+01,
         2.45744106e+01, -1.81966520e+00, -4.50436822e+00, -1.41094762e+01,
         1.50729312e+01,  8.08783460e+00, 

In [94]:
# 计算目标向量和数据集向量之间的距离
def most_similar_reviews(review):
    v = make_sent_vect(review)
    scores = Counter()
    # 点积计算相似度
    similar = np.dot(review2vec, v)
    # 记录句子的下标和相似度
    for i, sim in enumerate(similar):
        scores[i] = sim
        pass
    most_sim = list()
    for i, score in scores.most_common(3):
        most_sim.append(raw_reviews[i][0:40])
        pass
    return most_sim

In [95]:
most_similar_reviews(['boring', 'awful'])

['plot is not worth discussion even if it ',
 'i absolutely loved this film  i do not a',
 'absolutely awful movie . utter waste of ']

## 学习转移矩阵，创建有用的句子向量

### 正向传播

In [4]:
import numpy as np
def softmax(x_):
    x = np.atleast_2d(x_)
    temp = np.exp(x)
    return temp / np.sum(temp, axis=1, keepdims=True)

word_vects = {}
word_vects['yankees'] = np.array([[0.,0.,0.]])
word_vects['bears'] = np.array([[0.,0.,0.]])
word_vects['braves'] = np.array([[0.,0.,0.]])
word_vects['red'] = np.array([[0.,0.,0.]])
word_vects['socks'] = np.array([[0.,0.,0.]])
word_vects['lose'] = np.array([[0.,0.,0.]])
word_vects['defeat'] = np.array([[0.,0.,0.]])
word_vects['beat'] = np.array([[0.,0.,0.]])
word_vects['tie'] = np.array([[0.,0.,0.]])

sent2output = np.random.rand(3,len(word_vects))

identity = np.eye(3)

In [7]:
layer_0 = word_vects['red']
layer_1 = layer_0.dot(identity) + word_vects['socks']
layer_2 = layer_1.dot(identity) + word_vects['defeat']

print(layer_2)
print(np.dot(layer_2, sent2output))
pred = softmax(layer_2.dot(sent2output))
print(pred)

[[0. 0. 0.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
  0.11111111 0.11111111 0.11111111]]


### 反向传播

In [8]:
y = np.array([1,0,0,0,0,0,0,0,0])

pred_delta = pred - y
layer_2_delta = pred_delta.dot(sent2output.T)
defeat_delta = layer_2_delta * 1
layer_1_delta = layer_2_delta.dot(identity.T)
socks_delta = layer_1_delta * 1
layer_0_delta = layer_1_delta.dot(identity.T)
alpha = 0.01

word_vects['red'] -= layer_0_delta * alpha
word_vects['socks'] -= socks_delta * alpha
word_vects['defeat'] -= defeat_delta * alpha

identity -= np.outer(layer_0, layer_1_delta) * alpha
identity -= np.outer(layer_1, layer_2_delta) * alpha

sent2output -= np.outer(layer_2, pred_delta) * alpha

In [11]:
import sys,random,math
from collections import Counter
import numpy as np

f = open('tasksv11/en/qa1_single-supporting-fact_train.txt','r')
raw = f.readlines()
f.close()

tokens = list()
for line in raw[0:1000]:
    tokens.append(line.lower().replace("\n","").split(" ")[1:])

print(tokens[0:3])

[['mary', 'moved', 'to', 'the', 'bathroom.'], ['john', 'went', 'to', 'the', 'hallway.'], ['where', 'is', 'mary?', '\tbathroom\t1']]


In [12]:
vocab = set()
for sent in tokens:
    for word in sent:
        vocab.add(word)

vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i
    
def words2indices(sentence):
    idx = list()
    for word in sentence:
        idx.append(word2index[word])
    return idx

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [13]:
np.random.seed(1)
embed_size = 10

# word embeddings
embed = (np.random.rand(len(vocab),embed_size) - 0.5) * 0.1

# embedding -> embedding (initially the identity matrix)
recurrent = np.eye(embed_size)

# sentence embedding for empty sentence
start = np.zeros(embed_size)

# embedding -> output weights
decoder = (np.random.rand(embed_size, len(vocab)) - 0.5) * 0.1

# one hot lookups (for loss function)
one_hot = np.eye(len(vocab))

In [14]:
def predict(sent):
    
    layers = list()
    layer = {}
    layer['hidden'] = start
    layers.append(layer)

    loss = 0

    # forward propagate
    preds = list()
    for target_i in range(len(sent)):

        layer = {}

        # try to predict the next term
        layer['pred'] = softmax(layers[-1]['hidden'].dot(decoder))

        loss += -np.log(layer['pred'][sent[target_i]])

        # generate the next hidden state
        layer['hidden'] = layers[-1]['hidden'].dot(recurrent) + embed[sent[target_i]]
        layers.append(layer)

    return layers, loss

In [19]:
# forward
for iter in range(30000):
    alpha = 0.001
    sent = words2indices(tokens[iter%len(tokens)][1:])
    layers,loss = predict(sent) 

    # back propagate
    for layer_idx in reversed(range(len(layers))):
        layer = layers[layer_idx]
        target = sent[layer_idx-1]

        if(layer_idx > 0):  # if not the first layer
            layer['output_delta'] = layer['pred'] - one_hot[target]
            new_hidden_delta = layer['output_delta'].dot(decoder.transpose())

            # if the last layer - don't pull from a later one becasue it doesn't exist
            if(layer_idx == len(layers)-1):
                layer['hidden_delta'] = new_hidden_delta
            else:
                layer['hidden_delta'] = new_hidden_delta + layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
        else: # if the first layer
            layer['hidden_delta'] = layers[layer_idx+1]['hidden_delta'].dot(recurrent.transpose())
# update weights
    start -= layers[0]['hidden_delta'] * alpha / float(len(sent))
    for layer_idx,layer in enumerate(layers[1:]):
        
        decoder -= np.outer(layers[layer_idx]['hidden'], layer['output_delta']) * alpha / float(len(sent))
        
        embed_idx = sent[layer_idx]
        embed[embed_idx] -= layers[layer_idx]['hidden_delta'] * alpha / float(len(sent))
        recurrent -= np.outer(layers[layer_idx]['hidden'], layer['hidden_delta']) * alpha / float(len(sent))
        
    if(iter % 1000 == 0):
        print("Perplexity:" + str(np.exp(loss/len(sent))))

Perplexity:82.12126538007452
Perplexity:81.97370672664735
Perplexity:81.78264259039152
Perplexity:81.43451285094004
Perplexity:80.70574561844744
Perplexity:79.03101556936691
Perplexity:74.45141574632453
Perplexity:54.863481046626475
Perplexity:29.259868444646905
Perplexity:20.659460794962335
Perplexity:19.162678521697124
Perplexity:18.024594936900822
Perplexity:16.535656055558714
Perplexity:14.199333154141812
Perplexity:10.956862625435422
Perplexity:8.20744038147144
Perplexity:6.826254202085602
Perplexity:5.9779550022772945
Perplexity:5.406394170305721
Perplexity:5.082507506271119
Perplexity:4.870106090795196
Perplexity:4.73221369359872
Perplexity:4.649353606725815
Perplexity:4.589920992815768
Perplexity:4.525357199231706
Perplexity:4.445240492619834
Perplexity:4.351792505104332
Perplexity:4.251596413722791
Perplexity:4.15052196853435
Perplexity:4.047550228069106
